In [ ]:
# Lab Task 1 – Hough Transform for Line Detection in the m–c Domain_____________
# Implement the Hough Transform in the slope-intercept form of a line (y = mx + c).
# Steps:
# 1. Load a binary edge image (e.g., from Canny detector).
# 2. Define parameter space: slope m (finite range, e.g. -1 to 1) and intercept c.
# 3. Build the accumulator array by iterating over edge points and voting in (m, c) space.
# 4. Extract peaks in the accumulator to identify dominant lines.
# 5. Overlay the detected lines on the original image.

import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("building.jpg", cv2.IMREAD_GRAYSCALE)

    
edges_cv = cv2.Canny(img, 50, 150)

def hough_transform_mc(edges,step_size=1):
    
    h, w = edges.shape
    
   
    m_range = (-2, 2)  
    c_range = (-h, h + w) 

    step_size = 0.01
    bins = int((m_range[1] - m_range[0]) / step_size)
    m_bins = bins
    c_bins = int((c_range[1] - c_range[0]) / step_size)
    print(f"m_bins: {m_bins}, c_bins: {c_bins}")

    # Create parameter arrays
    m_values = np.linspace(m_range[0], m_range[1], m_bins)
    c_values = np.linspace(c_range[0], c_range[1], c_bins)
    
    # Step 3: Initialize accumulator array
    accumulator = np.zeros((m_bins, c_bins), dtype=np.int32)
    
    # Get edge pixel coordinates
    edge_points = np.where(edges > 0)
    y_coords, x_coords = edge_points
    
    print(f"Found {len(y_coords)} edge points")
    
    # Vote in parameter space
    for i in range(len(y_coords)):
        x, y = x_coords[i], y_coords[i]
        
        # For each slope value
        for m_idx, m in enumerate(m_values):
            # Calculate corresponding intercept: c = y - m*x
            c = y - m * x
            
            # Find closest c bin
            c_idx = np.argmin(np.abs(c_values - c))
            
            # Vote in accumulator
            accumulator[m_idx, c_idx] += 1
    
    return accumulator, m_values, c_values

In [ ]:
# Step 4: Run the Hough Transform and extract peaks
accumulator, m_values, c_values = hough_transform_mc(edges_cv)

def find_peaks(accumulator, threshold_ratio=0.5, min_distance=10):
    """
    Find peaks in the accumulator array
    """
    # Set threshold as a fraction of maximum value
    max_votes = np.max(accumulator)
    threshold = threshold_ratio * max_votes
    
    print(f"Max votes: {max_votes}, Threshold: {threshold}")
    
    # Find all points above threshold
    peak_indices = np.where(accumulator > threshold)
    peaks = list(zip(peak_indices[0], peak_indices[1]))
    
    # Sort by vote count (descending)
    peak_votes = [accumulator[m_idx, c_idx] for m_idx, c_idx in peaks]
    sorted_peaks = sorted(zip(peaks, peak_votes), key=lambda x: x[1], reverse=True)
    
    # Apply non-maximum suppression (simple version)
    filtered_peaks = []
    for (m_idx, c_idx), votes in sorted_peaks:
        # Check if this peak is too close to existing peaks
        too_close = False
        for existing_m, existing_c, _ in filtered_peaks:
            if abs(m_idx - existing_m) < min_distance and abs(c_idx - existing_c) < min_distance:
                too_close = True
                break
        
        if not too_close:
            filtered_peaks.append((m_idx, c_idx, votes))
            
        # Limit to top 10 lines
        if len(filtered_peaks) >= 10:
            break
    
    return filtered_peaks

# Find peaks
peaks = find_peaks(accumulator, threshold_ratio=0.3)
print(f"Found {len(peaks)} line candidates")

# Convert peak indices to parameter values
detected_lines = []
for m_idx, c_idx, votes in peaks:
    m = m_values[m_idx]
    c = c_values[c_idx]
    detected_lines.append((m, c, votes))
    print(f"Line: m={m:.3f}, c={c:.1f}, votes={votes}")

# Visualize accumulator space
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(edges_cv, cmap='gray')
plt.title('Edge Detection (Canny)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(accumulator, cmap='hot', aspect='auto', extent=[c_values[0], c_values[-1], m_values[-1], m_values[0]])
plt.colorbar(label='Votes')
plt.xlabel('Intercept (c)')
plt.ylabel('Slope (m)')
plt.title('Hough Space (m-c)')
# Mark detected peaks
for m_idx, c_idx, votes in peaks:
    plt.plot(c_values[c_idx], m_values[m_idx], 'wo', markersize=8, markeredgecolor='black')

plt.tight_layout()
plt.show()

In [ ]:
# Step 5: Overlay detected lines on the original image
def draw_lines_mc(image, lines, color=(0, 255, 0), thickness=2):
    """
    Draw lines in m-c form on the image
    """
    result = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) if len(image.shape) == 2 else image.copy()
    h, w = image.shape[:2]
    
    for m, c, votes in lines:
        # Calculate line endpoints
        # For x=0: y = c
        # For x=w-1: y = m*(w-1) + c
        
        x1, y1 = 0, int(c)
        x2, y2 = w-1, int(m * (w-1) + c)
        
        # Clip line to image boundaries
        if 0 <= y1 <= h-1 and 0 <= y2 <= h-1:
            cv2.line(result, (x1, y1), (x2, y2), color, thickness)
        elif y1 < 0 and y2 >= 0:  # Line enters from top
            # Find intersection with top edge (y=0)
            x_top = int(-c / m) if m != 0 else x1
            if 0 <= x_top <= w-1:
                cv2.line(result, (x_top, 0), (x2, y2), color, thickness)
        elif y1 <= h-1 and y2 > h-1:  # Line exits from bottom
            # Find intersection with bottom edge (y=h-1)
            x_bottom = int((h-1-c) / m) if m != 0 else x2
            if 0 <= x_bottom <= w-1:
                cv2.line(result, (x1, y1), (x_bottom, h-1), color, thickness)
        elif y1 < 0 and y2 > h-1:  # Line crosses entire image vertically
            x_top = int(-c / m) if m != 0 else 0
            x_bottom = int((h-1-c) / m) if m != 0 else w-1
            if 0 <= x_top <= w-1 and 0 <= x_bottom <= w-1:
                cv2.line(result, (x_top, 0), (x_bottom, h-1), color, thickness)
    
    return result

# Draw detected lines
result_image = draw_lines_mc(img, detected_lines)

# Display results
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(edges_cv, cmap='gray')
plt.title('Edge Detection')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(result_image)
plt.title(f'Detected Lines ({len(detected_lines)} lines)')
plt.axis('off')

plt.tight_layout()
plt.show()

# Compare with OpenCV's built-in Hough Line Transform
lines_cv = cv2.HoughLines(edges_cv, 1, np.pi/180, threshold=50)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(result_image)
plt.title('Our Implementation (m-c space)')
plt.axis('off')

# Draw OpenCV lines
if lines_cv is not None:
    result_cv = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    for line in lines_cv:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(result_cv, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    plt.subplot(1, 2, 2)
    plt.imshow(result_cv)
    plt.title(f'OpenCV Implementation (ρ-θ space) - {len(lines_cv)} lines')
    plt.axis('off')
else:
    plt.subplot(1, 2, 2)
    plt.imshow(img, cmap='gray')
    plt.title('OpenCV: No lines detected')
    plt.axis('off')

plt.tight_layout()
plt.show()